In [157]:
import pandas as pd
import streamlit
import itertools 
import logging

key = pd.read_csv("data/manual_key.txt",header=0, sep="\t",  quoting=3)
key = key.applymap(lambda row: row.replace('"',""))
key = key.applymap(lambda row: row.replace(' ',""))
source=pd.read_csv("data/source_sheet2.csv",header=0,  quoting=3)
#print(key)
#print(source)
pgp_combos_pre = key.values.tolist()
pgp_combos = [tuple(x) for x in key.values.tolist()]
pgp_flat_combos = [proguide for sublist in pgp_combos for proguide in sublist]
pgp_flat_combos_split = [proguide for sublist in pgp_flat_combos for proguide in sublist.split(",")]

target_combos = [tuple(x) for x in source.values.tolist()]
target_flat_combos = [target for sublist in target_combos for target in sublist]

max_steps= len((key.columns))
num_pgps= len(set(pgp_flat_combos_split))
num_cellgos = len(key.index)
pgp_combos_fixed=[]
for element in pgp_combos_pre:
    the_list=[map(lambda x: x.strip(), item.split(',')) for item in element]
    my_tuple=tuple(item for sub_list in the_list for item in sub_list)
    pgp_combos_fixed.append(my_tuple)
pgp_combos_fixed_lengths= [len(item) for item in pgp_combos_fixed]
wells = [f'{letter}{i}'for letter in ['A','B','C','D','E','F']  for i in range(1, 13)]
well_rows = list(itertools.chain.from_iterable([itertools.repeat(item, count) for item, count in zip(wells[0:num_cellgos],pgp_combos_fixed_lengths)]))
df = pd.DataFrame(columns=["SourceSite","SourceWell","TargetSite","TargetWell",'pgp'])
df['TargetWell'] = well_rows
df['TargetSite'] = 3
df['SourceSite'] = 1
df['pgp'] = pgp_flat_combos_split
idx = source.reset_index().set_index('pgp').loc[df.pgp, 'index'].values.tolist()
df['SourceWell']=source.well[idx].tolist()
df = pd.merge(left=df,right=source[["pgp","target"]],how="left",on='pgp')
df.to_csv('manual_output.txt',sep=',',index=False,quoting=None)
df.to_csv('output.txt',sep=',',index=False,quoting=None)
pgp_table = df.pgp.value_counts()
pgp_max=pgp_table.max()
pgp_min=pgp_table.min()
max_pgp_list=pgp_table[pgp_table == pgp_max].index.tolist()
min_pgp_list=pgp_table[pgp_table == pgp_min].index.tolist()
num_targets= len(set(df.target))
pipets_needed =len(df.index)
print(pgp_table)
with open('manual.log', 'w'):
   pass
logging.basicConfig(filename='manual.log',
                            filemode='w',
                            format='%(asctime)s,%(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.DEBUG,force=True)
logging.info(f'Max number of unique cellgorithm steps is {max_steps}')
logging.info(f'Number of proguides used is {num_pgps}')
logging.info(f'Number of unique targets is {num_targets}')
logging.info(f'Number of unique cellgos|unique TargetWells is {num_cellgos}')
logging.info(f'Total number of pipets needed/robotic transfers is {pipets_needed}')
logging.info(f'{max_pgp_list} proguide(s) use the most material. Each proguide(s) requires {pgp_max} transfers and {4  * pgp_max} ul minimum in the source plate (@ 4ul per transfer)')
logging.info(f'{min_pgp_list} proguide(s) use the least material require {pgp_min} transfers and {4 * pgp_min} ul minimum in the source plate')

PGP001    5
PGP002    5
PGP003    4
PGP004    4
PGP005    4
PGP006    4
PGP007    2
PGP008    2
Name: pgp, dtype: int64


In [155]:
len(df.index)

30

In [156]:
num_cellgos

5